### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.regression = 1 # don't change this

In [ ]:
system.train_set_end = 0.7 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
system.regression_move_threshold = 1.0 # emits a trade order if the predicted move is bigger than this

### ------------------------------------------------------------------------------------------------------------

### LinearRegression

In [ ]:
# Train LogisticRegression classifier on train data
reg, scaler = train_regressor(LinearRegression, data, plot_dist=1)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### XGBRegressor

In [ ]:
# Train XGBClassifier classifier on train data
reg, scaler = train_regressor(XGBRegressor, data)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### LGBMRegressor

In [ ]:
# Train LGBMClassifier classifier on train data
reg, scaler = train_regressor(LGBMRegressor, data)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### RandomForestRegressor

In [ ]:
# Train RandomForestClassifier classifier on train data
reg, scaler = train_regressor(RandomForestRegressor, data, n_jobs=-1)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### ExtraTreesRegressor

In [ ]:
# Train ExtraTreesClassifier classifier on train data
reg, scaler = train_regressor(ExtraTreesRegressor, data)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### CatBoostRegressor

In [ ]:
# Train CatBoostClassifier classifier on train data
reg, scaler = train_regressor(CatBoostRegressor, data, verbose=False)

In [ ]:
# Test on val data
*_, trades = rbacktest(reg, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------